El objetivo es encotrar la informacion mas posible, y que sea de calidad. De acuerdo a los criterios de PLD

| Información requerida                   | Información obtenida                              |
|-----------------------------------------|--------------------------------------------------|
| negocio                                 | MECA.CAPF_CLIENTES_POR_AREA relacionar con esta tabla |
| nombre cliente o razón social           | CLI_NOMBRE                                       |
| número cliente                          | CLI_CLAVE                                        |
| estatus                                 | CLI_EST_CLAVE                                    |
| número contrato                         |                                                  |
| género                                  |     N/A                                             |
| fecha_nacimiento o de constitución      |                                                  |
| entidad federativa nacimiento           |                                                  |
| país de nacimiento                      |                                                  |
| nacionalidad                            | CLI_NACIONALIDAD                                 |
| profesión                               |                                                  |
| calle                                   | CLI_DIRECCION                                    |
| número_exterior                         |                                                  |
| número_interior                         |                                                  |
| colonia_urbanización                    |                                                  |
| delegación_municipio                    |                                                  |
| ciudad_población                        |                                                  |
| entidad_federativa                      | CLI_EDO_CLAVE                                    |
| código postal                           |                                                  |
| país                                    | CLI_EDO_PAI_CLAVE                                |
| teléfono                                | CLI_TELEFONO                                     |
| correo_electrónico                      |                                                  |
| rfc                                     | CLI_RFC_LETRAS, CLI_RFC_FECHA, CLI_RFC_HOMO      |
| curp                                    |                                                  |
| fiel                                    |                                                  |
| representante legal                     |                                                  |
| tipo de persona                         | CLI_TDC_CLAVE                                    |
| clasificación por grado de riesgo       |                                                  |
| pep (persona políticamente expuesta)    |                                                  |
| razón                                   |                                                  |
| producto contratado                     | por movimientos y operaciones                    |
| cambios en perfil transaccional         |                                                  |

In [1]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
from rapidfuzz import process, fuzz
import numpy as np

In [44]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/ETL/CLientesT_FINAL_ACTIVOS.csv")

In [45]:
df["GENERO"] = df.apply(
    lambda row: "NO APLICA" if row["TIPO_PERSONA"] != "PF" and pd.isna(row["GENERO"]) 
    else row["GENERO"],
    axis=1
)

df["GENERO"] = df["GENERO"].apply(lambda x: "NO APLICA" if x=="NO_APLICA" else x)


In [46]:
df["NOMBRE_O_RAZON_SOCIAL"] = df["NOMBRE_O_RAZON_SOCIAL"].fillna("NO SE TIENE REGISTRADA LA INFORMACION")
df["TIPO_PERSONA"] = df["TIPO_PERSONA"].fillna("SIN INFO")
df["GENERO"] = df["GENERO"].fillna("SIN INFO")
df["RFC"] = df["RFC"].fillna("SIN INFO")
df["CURP"] = df["CURP"].fillna("SIN INFO")
df["FIEL"] = df["FIEL"].fillna("SIN INFO")
df["CALLE"] = df["CALLE"].fillna("NO SE TIENE REGISTRADA LA INFORMACION")
df["TELEFONO"] = df["TELEFONO"].fillna("NO SE TIENE REGISTRADA LA INFORMACION")
df["FECHA_NAC_O_CONST"] = df["FECHA_NAC_O_CONST"].fillna("NO SE TIENE REGISTRADA LA INFORMACION")
df["CORREO_ELECTRONICO"] = df["CORREO_ELECTRONICO"].fillna("NO SE TIENE REGISTRADA LA INFORMACION")
df["REPRESENTANTE_LEGAL"] = df["REPRESENTANTE_LEGAL"].fillna("NO SE TIENE REGISTRADA LA INFORMACION")


In [56]:
df[df.duplicated(subset=["NOMBRE_O_RAZON_SOCIAL"], keep="first")]

,ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,...,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
658,656,NaN,APOYO ECONOMICO FAMILIAR SA DE CV SOFOM ENR,22336018,VIGENTE,39353468,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,NaN,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,AEF050221TY2,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,NaN,SIRAC,10/12/24
945,938,NaN,"BANCA AFIRME, S.A.",1041222,VIGENTE,NaN,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,NaN,...,,,BAF950102JP5,SIN INFO,SIN INFO,ARZATE ORTIZ RAUL ROSENDO -- ASMITIA ANCONA JO...,Moral Nacional no Gravable,NaN,TAS,10/12/24
946,940,NaN,"BANCA AFIRME, S.A.","306, 1041222",VIGENTE,NaN,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,NaN,...,,,BAF950102JP5,SIN INFO,SIN INFO,ARZATE ORTIZ RAUL ROSENDO -- ASMITIA ANCONA JO...,"PM, Moral Nacional no Gravable",NaN,"SIPE, TAS",10/12/24
952,946,NaN,"BANCA CREMI,S.A.","BCDI, 77",VIGENTE,NaN,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,NaN,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,NaN,"SIPE, MECA",10/12/24
976,966,NaN,BANCO AUTOFIN MEXICO S.A.,1059997,VIGENTE,NaN,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,NaN,...,56 15 03 37,,BAM0511076B3,SIN INFO,SIN INFO,ALVAREZ SANCHEZ ANDRES LEONEL,Moral Nacional no Gravable,NaN,TAS,10/12/24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11254,11102,NaN,UNION BANK OF SWITZERLAND,5,VIGENTE,NaN,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,NaN,...,800 813 9224,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,NaN,SIPE,10/12/24
11255,11103,NaN,UNION BANK OF SWITZERLAND,"5, 10459",VIGENTE,NaN,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,NaN,...,800 813 9224,NO SE TIENE REGISTRADA LA INFORMACION,XXXXXXXXXXXXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,NaN,"SIPE, SOI",10/12/24
11309,11157,UNIDAD DE MERCADO DE CAPITALES,"VALORES MEXICANOS CASA DE BOLSA, S.A. DE C.V.",VALMEX,VIGENTE,NaN,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,NaN,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,XXX123456XXX,SIN INFO,SIN INFO,NO SE TIENE REGISTRADA LA INFORMACION,PM,NaN,MECA,10/12/24
11310,11158,UNIDAD DE MERCADO DE CAPITALES,"VALORES MEXICANOS CASA DE BOLSA, S.A. DE C.V.","VALMEX, 1057519, 315",VIGENTE,NaN,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,NaN,...,", 566 6211, 5279 1200",,"XXX123456XXX, CBC79081421A, VMC930101JS6",SIN INFO,SIN INFO,ALVAREZ TOLEDO CLAUDIA CECILIA -- PAYAN LOPEZ ...,"PM, Moral Nacional no Gravable",NaN,"MECA, TAS",10/12/24


In [ ]:
columnas = [
    "NUMERO_CLIENTE",
    "NOMBRE_O_RAZON_SOCIAL",
    "ESTATUS",
    "TIPO_PERSONA",
    "GENERO",
    "RFC",
    "CURP",
    "FIEL",
    "CALLE",
    "TELEFONO",
    "FECHA_NAC_O_CONST",
    "CORREO_ELECTRONICO",
    "REPRESENTANTE_LEGAL"
]

In [ ]:
data = df[df["ID_DL"]==1]

In [53]:
data

,ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,...,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
0,1,NaN,DOMINION INDUSTRIAL S.A DE C.V,3620200,VIGENTE,40681346,NO APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NaN,MEXICO,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,DIN930813DV4,SIN INFO,SIN INFO,EDMUNDO LOPEZ GALICIA,PM,NaN,SIRAC,10/12/24


In [ ]:
df[df["ID_DL"] == client_id]

In [10]:
df = df.fillna("INF NO DISP")

,ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,...,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
0,1,NO SE TIENE REGISTRADA LA INFORMACION,DOMINION INDUSTRIAL S.A DE C.V,3620200,VIGENTE,40681346,NO_APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,MEXICO,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,DIN930813DV4,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,EDMUNDO LOPEZ GALICIA,PM,NO SE TIENE REGISTRADA LA INFORMACION,SIRAC,10/12/24
1,2,NO SE TIENE REGISTRADA LA INFORMACION,NAFIN FIDUCIARIO FIDEICOMISO CONTRAGARANTIA F...,1063586,VIGENTE,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,...,5325 6914,,NFI3406305T0,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,GUERRERO HERRERA RAFAEL GIL,Moral Nacional no Gravable,NO SE TIENE REGISTRADA LA INFORMACION,TAS,10/12/24
2,3,NO SE TIENE REGISTRADA LA INFORMACION,"PROVEEDOR DE TRACTO REFACCIONES, S. A. DE C. V.",28316118,VIGENTE,40628148,NO_APLICA,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,MEXICO,...,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,PTR980414UD4,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,REYES FLORES VELAZQUEZ,PM,NO SE TIENE REGISTRADA LA INFORMACION,SIRAC,10/12/24
3,4,NO SE TIENE REGISTRADA LA INFORMACION,".+ TASA ACTIVO, S.A. DE C.V., F.I.R.V.",1064707,VIGENTE,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,...,55 5033 3341,,STA130219T22,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,MARQUEZ LARA RENE ALONSO,Moral Nacional no Gravable,NO SE TIENE REGISTRADA LA INFORMACION,TAS,10/12/24
4,5,NO SE TIENE REGISTRADA LA INFORMACION,".+TASA LARGO PLAZO, S.A. DE C.V., F.I.I.D.",1064327,VIGENTE,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,...,55 5033 3341,,STL100514QG9,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,MARQUEZ LARA RENE ALONSO,Moral Nacional no Gravable,NO SE TIENE REGISTRADA LA INFORMACION,TAS,10/12/24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11629,11473,NO SE TIENE REGISTRADA LA INFORMACION,"ZURICH SANTANDER SEGUROS M�XICO, S.A.",1064521,VIGENTE,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,...,55 5169 4300,,SSG971008QUI,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,CUENCA KYFF CLAUDIA -- ROJO PALAVICINI TANIA A...,Moral Nacional no Gravable,NO SE TIENE REGISTRADA LA INFORMACION,TAS,10/12/24
11630,11474,NO SE TIENE REGISTRADA LA INFORMACION,"ZURICH VIDA COMPA�IA DE SEGUROS, S.A.","1064384, 6009265",VIGENTE,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,...,"55 5284 1000, 55 5284 1000 EXT.520744",,ZVS941209AH6,NO SE TIENE REGISTRADA LA INFORMACION,NO SE TIENE REGISTRADA LA INFORMACION,DEL R�O . CRISTIAN ALBERTO -- F� RIVERA RICARD...,Moral Nacional no Gravable,NO SE TIENE REGISTRADA LA INFORMACION,TAS,10/12/24
11631,11475,NO SE TIENE REGISTRADA LA INFORMACION,ZW RECICLAJE SA DE CV,28852888,VIGENTE,40620974,NO_APLICA,NO SE TIENE

In [9]:
df[df["ID_DL"] == 1]

,ID_DL,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,...,TELEFONO,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
0,1,NaN,DOMINION INDUSTRIAL S.A DE C.V,3620200,VIGENTE,40681346,NO_APLICA,NaN,NaN,MEXICO,...,NaN,NaN,DIN930813DV4,NaN,NaN,EDMUNDO LOPEZ GALICIA,PM,NaN,SIRAC,10/12/24


In [8]:
df[df["ID_DL"] == 1].to_json()

'{"ID_DL":{"0":1},"NEGOCIO":{"0":null},"NOMBRE_O_RAZON_SOCIAL":{"0":" DOMINION INDUSTRIAL S.A DE C.V  "},"NUMERO_CLIENTE":{"0":"3620200"},"ESTATUS":{"0":"VIGENTE"},"NUMERO_CONTRATO":{"0":"40681346"},"GENERO":{"0":"NO_APLICA"},"FECHA_NAC_O_CONST":{"0":null},"ENTIDAD_FEDERATIVA_NACIMIENTO":{"0":null},"PAIS_DE_NACIMIENTO":{"0":"MEXICO"},"NACIONALIDAD":{"0":"MEXICANA"},"PROFESION":{"0":null},"CALLE":{"0":null},"NUMERO_EXTERIOR":{"0":null},"NUMERO_INTERIOR":{"0":null},"COLONIA_URBANIZACION":{"0":null},"DELEGACION_MUNICIPIO":{"0":null},"CIUDAD_POBLACION":{"0":null},"ENTIDAD_FEDERATIVA":{"0":null},"CODIGO_POSTAL":{"0":null},"PAIS":{"0":"MEXICO"},"TELEFONO":{"0":null},"CORREO_ELECTRONICO":{"0":null},"RFC":{"0":"DIN930813DV4"},"CURP":{"0":null},"FIEL":{"0":null},"REPRESENTANTE_LEGAL":{"0":"EDMUNDO LOPEZ GALICIA"},"TIPO_PERSONA":{"0":"PM"},"PRODUCTO_CONTRATADO":{"0":null},"SISTEMA_ORIGEN":{"0":"SIRAC"},"FECHA_DE_ACTUALIZACION":{"0":"10\\/12\\/24"}}'

In [114]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/Clientes_activos_MECA.dat")

## NEGOCIO
SE ENCONTRO UNA POSIBLE RELACION CON RESPECTO A NEGOCIO POR CATALOGO DE CLIENTE_POR_AREA TODOS RELACIONADOS A UNIDAD DE MERCADO DE CAPITALES, SE HACE UNA ESTANDARIZACION POR MAYUSCULA Y MAPEANDO VALORES NULOS A NO SE ENCONTRO INFORMACION

In [29]:
df.value_counts("NEGOCIO")

NEGOCIO
UNIDAD DE MERCADO DE CAPITALES                                          49
SUBDIRECCION DE MERCADO DE CAPITALES, UNIDAD DE MERCADO DE CAPITALES     1
Name: count, dtype: int64

In [30]:
df["NEGOCIO"] = df["NEGOCIO"].apply(lambda x: x.upper() if pd.notna(x) else "NO SE ENCONTRO INFORMACION")

In [31]:
df.value_counts("NEGOCIO")

NEGOCIO
NO SE ENCONTRO INFORMACION                                              213
UNIDAD DE MERCADO DE CAPITALES                                           49
SUBDIRECCION DE MERCADO DE CAPITALES, UNIDAD DE MERCADO DE CAPITALES      1
Name: count, dtype: int64

In [115]:
df = df.apply(lambda x: x.upper() if isinstance(x, str) else x)

In [40]:
df

,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,NACIONALIDAD,...,CLI_RFC_LETRAS,CLI_RFC_FECHA,CLI_RFC_HOMO,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,TIPO_DE_CLIENTE,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN
0,NO SE ENCONTRO INFORMACION,ABA DIVISAS CASA DE CAMBIO SA CV (GUAD),ABADIV,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEX,...,NaN,NaN,NaN,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA
1,NO SE ENCONTRO INFORMACION,"ABA DIVISAS CASA DE CAMBIO, SA DE CV",ABA,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEX,...,NaN,NaN,NaN,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA
2,UNIDAD DE MERCADO DE CAPITALES,ACCIONES Y VALORES DE MEXICO,ACCIV,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEX,...,AVM,760909.0,UK0,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA
3,UNIDAD DE MERCADO DE CAPITALES,ACTINVER CASA DE BOLSA,ACTINV,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEX,...,ACB,760907.0,6M2,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA
4,NO SE ENCONTRO INFORMACION,AMRO BANK MEXICO,AMRO,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEX,...,NaN,NaN,NaN,NaN,NaN,NaN,PM,BANCO,NaN,MECA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,NO SE ENCONTRO INFORMACION,"VECTODIVISAS CASA DE CAMBIO,SA DE CV",VECTO,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEX,...,NaN,NaN,NaN,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA
259,UNIDAD DE MERCADO DE CAPITALES,"VECTOR, CASA DE BOLSA, S.A DECV",VECT,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEX,...,XXX,123456.0,XXX,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA
260,NO SE ENCONTRO INFORMACION,VECTORMEX,VECTOR,VIGENTE,NaN,NaN,NaN,NaN,NaN,EXT,...,NaN,NaN,NaN,NaN,NaN,NaN,PM,BANCO,NaN,MECA
261,NO SE ENCONTRO INFORMACION,VER FCMX,FCHINY,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEX,...,NaN,NaN,NaN,NaN,NaN,NaN,PM,BANCO,NaN,MECA


## NOMBRE CLIENTE O RAZON SOCIAL

SE NORMALIZARA PASANDO A MAYUSCULA, DADO POR LA COMPLEJIDAD DEL NOMBRE QUE PUEDE CONTENER CARACTERES ESPECIALES SE CONSERVA EL NOMBRE COMO VIENE DE LA BASE DE DATOS, SOLO SE QUITARA VALORES ATIPICOS (OUTLIERS)


In [32]:
# Supongamos que df es tu DataFrame original
df_conteo = df[["NUMERO_CLIENTE", "NOMBRE_O_RAZON_SOCIAL"]]
df_conteo["SIZE_NAME"] = df_conteo["NOMBRE_O_RAZON_SOCIAL"].apply(lambda x: len(x))

# Calcular los cuartiles y el rango intercuartílico (IQR)
Q1 = df_conteo["SIZE_NAME"].quantile(0.25)
Q3 = df_conteo["SIZE_NAME"].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites para los valores atípicos
limite_inferior = Q1 - IQR
limite_superior = Q3 + 1 * IQR

# Filtrar los valores atípicos
valores_atipicos = df_conteo[(df_conteo["SIZE_NAME"] < limite_inferior) | (df_conteo["SIZE_NAME"] > limite_superior)]

print(valores_atipicos.to_string(max_cols=None))

    NUMERO_CLIENTE                                          NOMBRE_O_RAZON_SOCIAL  SIZE_NAME
90             BTG                BTG PACTUAL MEXICO, CASA DE BOLSA, S.A. DE C.V.         47
99          CREDIS             CASA DE BOLSA CREDIT SUISSE (MÉXICO), S.A. DE C.V.         50
100         MULTIV   CASA DE BOLSA MULTIVA, S.A. DE C.V. GRUPO FINANCIERO MULTIVA         60
157           HSBC        HSBC CASA DE BOLSA, S.A. DE .C.V. GRUPO FINANCIERO HSBC         55
162         INGBAR  ING (MEXICO), S.A.DE C.V. CASA DE BOLSA, ING GRUPO FINANCIERO         61
190         MLYNCH               MERRILL LYNCH MEXICO, S.A. DE C.V. CASA DE BOLSA         48
252         UGATLX               UNION DE CREDITO AGRICOLA Y GANADERA DE TLAXCALA         48


/tmp/ipykernel_5804/2977704530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conteo["SIZE_NAME"] = df_conteo["NOMBRE_O_RAZON_SOCIAL"].apply(lambda x: len(x))


In [33]:
limite_inferior

4.0

In [10]:
df["NOMBRE_O_RAZON_SOCIAL"] = df["NOMBRE_O_RAZON_SOCIAL"].apply(lambda x: x.upper())

In [11]:
print(df.value_counts("NOMBRE_O_RAZON_SOCIAL").to_string(max_rows=None))

NOMBRE_O_RAZON_SOCIAL
CHASE MANHATTAN BANK N.Y.                                        3
BBV SECURITIES INC.                                              2
ABA DIVISAS CASA DE CAMBIO SA CV (GUAD)                          1
J.P. MORGAN, CASA DE BOLSA, S.A. DE C.V.                         1
LEHMAN BROS.INT'L EUROPE                                         1
LATINVEST SECURITIES                                             1
LATINVEST LONDRES                                                1
LATINVEST INTERNATIONAL                                          1
IXE CASA DE BOLSA, S.A. DE C.V.                                  1
LINDA DISTRIBUIDORA,S.A.(MONTERREY)                              1
IXE BANCO, S.A.                                                  1
INVEX, CASA DE BOLSA, S.A. DE C.V                                1
INVERSORA BURSATIL, S.A. DE C.V                                  1
INVERMEXICO CASA DE CAMBIO,S.A.                                  1
INVERMEXICO                             

In [12]:
print(df[df["NOMBRE_O_RAZON_SOCIAL"] == "OTRAS AREAS"].to_string(max_rows=None))

                        NEGOCIO NOMBRE_O_RAZON_SOCIAL NUMERO_CLIENTE  ESTATUS  NUMERO_CONTRATO  GENERO  FECHA_NAC_O_CONST  ENTIDAD_FEDERATIVA_NACIMIENTO  PAIS_DE_NACIMIENTO NACIONALIDAD  PROFESION                                  CALLE  NUMERO_EXTERIOR  NUMERO_INTERIOR  COLONIA_URBANIZACION  DELEGACION_MUNICIPIO  CIUDAD_POBLACION ENTIDAD_FEDERATIVA  CODIGO_POSTAL    PAIS TELEFONO  CORREO_ELECTRONICO  RFC CLI_RFC_LETRAS  CLI_RFC_FECHA CLI_RFC_HOMO  CURP  FIEL  REPRESENTANTE_LEGAL TIPO_PERSONA TIPO_DE_CLIENTE  PRODUCTO_CONTRATADO SISTEMA_ORIGEN
213  NO SE ENCONTRO INFORMACION           OTRAS AREAS          OTRAS  VIGENTE              NaN     NaN                NaN                            NaN                 NaN          MEX        NaN  NO SE TIENE REGISTRADA LA INFORMACION              NaN              NaN                   NaN                   NaN               NaN   DISTRITO FEDERAL            NaN  MEXICO      NaN                 NaN  NaN            NaN            NaN          NaN 

|              Valor atipico              |                     Conteo                       |
|-----------------------------------------|--------------------------------------------------|
| OTRAS AREAS                             |                         1                        |
| PARTES Y DISENOS NACIONALES, S.A.       |                         1                        |
| PERSONA FISICA                          |                         1                        |
| PERSONA FISICA GUADALAJARA              |                         1                        |
| PERSONA FISICA LEON                     |                         1                        |
| SUC.NUEVO LAREDO                        |                         1                        |
| SUCURSAL CUERNAVACA                     |                         1                        |
| SUCURSAL GUADALAJARA                    |                         1                        |
| SUCURSAL MONTERREY                      |                         1                        |
| SUCURSAL REYNOSA                        |                         1                        |
| AREA RECURSOS HUMANOS                   |                         1                        |
| ACCIONES Y VALORES DE MEXICO            |                         1                        |
| AREA CREDITO                            |                         1                        |
| AREA FIDEICOMISOS                       |                         1                        |
| AREA INTERNACIONAL                      |                         1                        |
| AREA MERCADO DE CAPITALES               |                         1                        |
| AREA MERCADO DE DINERO                  |                         1                        |

## ESTATUS
SE MAPEARA A VIGENTE O NO VIGENTE DE LA COLUMNA CLI_EST_CLAVE

In [42]:
df.value_counts("ESTATUS")

ESTATUS
VIGENTE    263
Name: count, dtype: int64

## NACIONALIDAD 
SE MAPEO MEX: MEXICANA, EXT: EXTRANJERA, EN CASO DE SER NULO SE "NO SE ENCONTRO INFORMACION" 

In [44]:
df.value_counts("NACIONALIDAD")

NACIONALIDAD
MEX    170
EXT     93
Name: count, dtype: int64

In [116]:
df["NACIONALIDAD"] = df["NACIONALIDAD"].apply(lambda x: "NO SE ENCONTRO INFORMACION" if pd.isna(x) else x)
df["NACIONALIDAD"] = df["NACIONALIDAD"].apply(lambda x: "MEXICANA" if x == "MEX" else x)
df["NACIONALIDAD"] = df["NACIONALIDAD"].apply(lambda x: "EXTRANJERA" if x == "EXT" else x)
df.value_counts("NACIONALIDAD")

NACIONALIDAD
MEXICANA      170
EXTRANJERA     93
Name: count, dtype: int64

## DIRECCION
ESTE CAMPO SE DISPERSARA COMO CALLE

In [117]:
palabras_no_validas = ["PENDIENTE", "JHGJHG", "NINGUNA", "TEL"]

df["CALLE"] = df["CALLE"].apply(
    lambda x: x if isinstance(x, str) and 
               not any(palabra in x for palabra in palabras_no_validas) and \
               x != 'P' and \
               not x.replace(" ", "").isdigit() 
               else np.nan
)

In [118]:
print(df.value_counts("CALLE").to_string(max_rows=None))

CALLE
NO SE TIENE REGISTRADA LA INFORMACION                                                                   172
SUC MONTERREY                                                                                             3
1345 AVENUE OF AMERICAS NEW YORK, N.Y. 10105                                                              2
SUC GUADALAJARA                                                                                           2
RUBEN DARIO 281, PISO 10                                                                                  2
1000 LOUSIANA,SUITE 6910                                                                                  1
PASEO DE LA REFORMA 116                                                                                   1
MANUEL AVILA CAMACHO 36 - PISO 22, COL. LOMAS DE CHAPULTEPEC, CP 11000, DEL. MIGUEL HIDALGO               1
MARINA NACIONAL                                                                                           1
MARIO                 

In [ ]:
df.value_counts("CLI_DIRECCION")

CLI_DIRECCION
NO SE TIENE REGISTRADA LA INFORMACION                                                      41757
BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.     3926
RIO AMAZONAS #91 COL. CUAUHTEMOC CP 06500                                                   1994
REFORMA #398 COL. JUAREZ PB CP 06600                                                         808
BOSQUE DE CIRUELOS 120, BOSQUES DE LAS LOMAS C.P. 11700, MEXICO, D.F.                        672
PASEO DE LA REFORMA #383 PISO 14 COL. CUAUHTEMOC CP 06500                                    464
INSURGENTES SUR 1886 COL.FLORIDA                                                              91
MONTES URALES #424 PISO 3, COLONIA LOMAS DE CHAPULTEPEC C.P. 11000                             4
INSURGENTES SUR 1971                                                                           3
Name: count, dtype: int64

In [ ]:
def separar_direccion(direccion):
    # Ejemplo simple de separación usando regex
    match = re.match(r'^(.*?)(\d+.*?)(COL\..*?)(CP \d+)(.*)$', direccion)
    if match:
        return match.groups()
    else:
        return (direccion, "", "", "", "")

In [75]:
separar_direccion("BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.")

('BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.',
 '',
 '',
 '',
 '')

In [94]:
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import TfidfVectorizer

# Datos de ejemplo
data = pd.DataFrame({
    'CLI_NOMBRE': ['P',
       'BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES POLANCO, MIGUEL HIDALGO, 11510 MEXICO, DF.',
       'NO SE TIENE REGISTRADA LA INFORMACION',
       'RIO AMAZONAS #91 COL. CUAUHTEMOC CP 06500',
       'REFORMA #398 COL. JUAREZ PB CP 06600',
       'BOSQUE DE CIRUELOS 120, BOSQUES DE LAS LOMAS C.P. 11700, MEXICO, D.F.',
       'PASEO DE LA REFORMA #383 PISO 14 COL. CUAUHTEMOC CP 06500',
       'INSURGENTES SUR 1886 COL.FLORIDA', 'JHGJHG', 'PENDIENTE',
       'MONTES URALES #424 PISO 3, COLONIA LOMAS DE CHAPULTEPEC C.P. 11000',
       'INSURGENTES SUR 1971']
})

# Vectorización TF-IDF
vectorizer = TfidfVectorizer()
encoded_data = vectorizer.fit_transform(data['CLI_NOMBRE']).toarray()

# Aplicar IsolationForest
model = IsolationForest(contamination=0.001)
model.fit(encoded_data)
outliers = model.predict(encoded_data)

# Identificar valores atípicos
data['ANOMALY'] = outliers
print(data[data['ANOMALY'] == -1])

                                          CLI_NOMBRE  ANOMALY
1  BLAS PASCAL NO. 205-2Ø PISO COL. LOS MORALES P...       -1


## CLI_TELEFONO

In [93]:
df["TELEFONO"].unique()

array([nan, '3 26 80 00', '3-26-48-48 JORGE BUSTOS',
       '52-82-62-00  FAX: 52-82-62-24', '2306333 5910011',
       'HOUSTON TEXAS', '95 212 7519602', '9136786541',
       '2088761, 5255654', '630-42-55', '2827800', '202 41 34 FAX',
       '212 836 5520', 'ELSA PEREZ', '2.59.47.29', '212 841 37 07',
       '52413232, 52413233', '(212) 7282343', '95 212 272 1000',
       '2576600', '52-59-47-74', '(5255) 5246-7022', '95212 922 39 22',
       '2-01-20-00', '(0155) 5625 1665, Cynthia Martinez Sol',
       '257-97 35 AL 44', '95 212 6384227', 'FAX 95 212 8346510',
       '280 08 42', '53-25-70-52', '95 212 325 2000', '95 212 2382411',
       '496991034978', '95 212 5042185 Y 2115', '212 504 2270', '5283000',
       '55409971', '283 33 00', '2826645', '581-53-93, 581-53-84',
       '5721-2222  fax 5721-5159', '2828900 Y 2803497', '2582180',
       '2-08-30-98 CARLOS TAPIA', '2021202, 2021092',
       '3-27-33-33 GENARO SOTO', '6252525', 'CODIGO DEAL LATL',
       '95 201 524 5516', '95 2

In [94]:
print(df.value_counts("TELEFONO").to_string(max_rows=None))

TELEFONO
(212) 7282343                             2
9136786541                                2
53-25-70-52                               2
581-53-93, 581-53-84                      1
95 201 524 5516                           1
6625507  FAX                              1
630-42-55                                 1
627-92-70                                 1
6252525                                   1
5721-2222  fax 5721-5159                  1
52-82-62-00  FAX: 52-82-62-24             1
5654                                      1
55409971                                  1
53-25-60-00                               1
5283000                                   1
52413232, 52413233                        1
95 212 2382411                            1
(0155) 5625 1665, Cynthia Martinez Sol    1
95 212 325 2000                           1
95 212 449 6777                           1
95 212 5042185 Y 2115                     1
95 212 5256494                            1
95 212 5744802         

In [106]:
df_duplicate = df[df["CLI_TELEFONO"].notna()].drop_duplicates("CLI_NOMBRE")

In [ ]:
df_duplicate

## RFC
ESTANDARIZACION DEL RFC GENERICO XXX-XXXXXX-XXX Y 
CON MAYUSCULAS Y CONCATENACION

In [119]:
# GUARDAMOS LAS COLUMANS
columnas =  df.columns

In [120]:
df["CLI_RFC_LETRAS"]= df["CLI_RFC_LETRAS"].apply(lambda x: x.upper() if pd.notna(x) else "XXX")
df["CLI_RFC_FECHA"] = df["CLI_RFC_FECHA"].apply(lambda x: int(x) if pd.notna(x) else "XXXXXX") 
df["CLI_RFC_HOMO"] = df["CLI_RFC_HOMO"].apply(lambda x: x.upper() if pd.notna(x) else "XXX") 
df["RFC"] = df.apply(lambda row: f'{row["CLI_RFC_LETRAS"]}{row["CLI_RFC_FECHA"]}{row["CLI_RFC_HOMO"]}',axis=1)


In [121]:
print(df.value_counts("RFC").to_string(max_rows=None))

RFC
XXXXXXXXXXXX     219
XXX123456XXX      24
ACB7609076M2       1
XXXX999999999      1
VFI770713PB8       1
VCB70518M88        1
PCB50107CP2        1
PCB111011816       1
NFI3406305TO       1
JPM960506G57       1
ICB8704225X7       1
111                1
HCB950125897       1
CCV40120SQA        1
CBC202214A6        1
CBB960628JY9       1
CBA8701315H8       1
BPC1310158C1       1
BCC6031613A        1
BCB712185M8        1
AVM760909UK0       1
ICB61106G80        1


In [122]:
df["RFC"] = df["RFC"].apply(lambda x: x if verificar_rfc(x) else "XXXXXXXXXXXXX")

In [123]:
new_columnas = [col for col in columnas if col not in ["CLI_RFC_LETRAS","CLI_RFC_FECHA","CLI_RFC_HOMO"]]

In [124]:
df = df[new_columnas]

In [ ]:
df

In [102]:
df[[]]

,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,NACIONALIDAD,...,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,TIPO_DE_CLIENTE,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,RFC
0,NaN,ABA DIVISAS CASA DE CAMBIO SA CV (GUAD),ABADIV,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,XXXXXXXXXXXXX
1,NaN,"ABA DIVISAS CASA DE CAMBIO, SA DE CV",ABA,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,XXXXXXXXXXXXX
2,UNIDAD DE MERCADO DE CAPITALES,ACCIONES Y VALORES DE MEXICO,ACCIV,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANA,...,NaN,AVM760909UK0,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,AVM760909UK0
3,UNIDAD DE MERCADO DE CAPITALES,ACTINVER CASA DE BOLSA,ACTINV,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANA,...,NaN,ACB7609076M2,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,ACB7609076M2
4,NaN,AMRO BANK MEXICO,AMRO,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,XXXXXXXXXXXXX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,NaN,"VECTODIVISAS CASA DE CAMBIO,SA DE CV",VECTO,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,XXXXXXXXXXXXX
259,UNIDAD DE MERCADO DE CAPITALES,"VECTOR, CASA DE BOLSA, S.A DECV",VECT,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANA,...,NaN,XXX123456XXX,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,XXX123456XXX
260,NaN,VECTORMEX,VECTOR,VIGENTE,NaN,NaN,NaN,NaN,NaN,EXTRANJERA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,XXXXXXXXXXXXX
261,NaN,VER FCMX,FCHINY,VIGENTE,NaN,NaN,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,XXXXXXXXXXXXX


In [120]:
df.value_counts("RFC")

RFC
XXXXXXXXXXXX    22036
XXX123456XXX    20399
ACB7609076M2     3926
VFI770713PB8     1994
AVM760909UK0      808
ICB8704225X7      464
111                85
CBB960628JY9        4
NFI3406305TO        3
Name: count, dtype: int64

In [125]:
df["GENERO"] =  "N/A"

In [104]:
df

,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,NACIONALIDAD,...,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,TIPO_DE_CLIENTE,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,RFC
0,NaN,ABA DIVISAS CASA DE CAMBIO SA CV (GUAD),ABADIV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,XXXXXXXXXXXXX
1,NaN,"ABA DIVISAS CASA DE CAMBIO, SA DE CV",ABA,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,XXXXXXXXXXXXX
2,UNIDAD DE MERCADO DE CAPITALES,ACCIONES Y VALORES DE MEXICO,ACCIV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,AVM760909UK0,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,AVM760909UK0
3,UNIDAD DE MERCADO DE CAPITALES,ACTINVER CASA DE BOLSA,ACTINV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,ACB7609076M2,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,ACB7609076M2
4,NaN,AMRO BANK MEXICO,AMRO,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,XXXXXXXXXXXXX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,NaN,"VECTODIVISAS CASA DE CAMBIO,SA DE CV",VECTO,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,XXXXXXXXXXXXX
259,UNIDAD DE MERCADO DE CAPITALES,"VECTOR, CASA DE BOLSA, S.A DECV",VECT,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXX123456XXX,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,XXX123456XXX
260,NaN,VECTORMEX,VECTOR,VIGENTE,NaN,N/A,NaN,NaN,NaN,EXTRANJERA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,XXXXXXXXXXXXX
261,NaN,VER FCMX,FCHINY,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,XXXXXXXXXXXXX


In [105]:
df

,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,NACIONALIDAD,...,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,TIPO_DE_CLIENTE,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,RFC
0,NaN,ABA DIVISAS CASA DE CAMBIO SA CV (GUAD),ABADIV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,XXXXXXXXXXXXX
1,NaN,"ABA DIVISAS CASA DE CAMBIO, SA DE CV",ABA,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,XXXXXXXXXXXXX
2,UNIDAD DE MERCADO DE CAPITALES,ACCIONES Y VALORES DE MEXICO,ACCIV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,AVM760909UK0,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,AVM760909UK0
3,UNIDAD DE MERCADO DE CAPITALES,ACTINVER CASA DE BOLSA,ACTINV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,ACB7609076M2,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,ACB7609076M2
4,NaN,AMRO BANK MEXICO,AMRO,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,XXXXXXXXXXXXX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,NaN,"VECTODIVISAS CASA DE CAMBIO,SA DE CV",VECTO,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,XXXXXXXXXXXXX
259,UNIDAD DE MERCADO DE CAPITALES,"VECTOR, CASA DE BOLSA, S.A DECV",VECT,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXX123456XXX,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,XXX123456XXX
260,NaN,VECTORMEX,VECTOR,VIGENTE,NaN,N/A,NaN,NaN,NaN,EXTRANJERA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,XXXXXXXXXXXXX
261,NaN,VER FCMX,FCHINY,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,XXXXXXXXXXXXX


"NEGOCIO, 
"NOMBRE_O_RAZON_SOCIAL,
"NUMERO_CLIENTE, 
"ESTATUS, 
"NÚMERO_CONTRATO, 
"GENERO,
"FECHA_NAC_O_CONST, 
"ENTIDAD_FEDERATIVA_NACIMIENTO, 
"PAIS_DE_NACIMIENTO, 
"NACIONALIDAD, 
"PROFESION, 
"CALLE, 
"NUMERO_EXTERIOR, 
"NUMERO_INTERIOR, 
"COLONIA_URBANIZACIÓN, 
"DELEGACION_MUNICIPIO, 
"CIUDAD_POBLACION, 
"ENTIDAD_FEDERATIVA, 
"CÓDIGO_POSTAL, 
"PAIS, 
"TELEFONO, 
"CORREO_ELECTRÓNICO, 
"RFC, 
"CURP, 
"FIEL, 
"REPRESENTANTE_LEGAL, 
"TIPO_PERSONA, 
"PRODUCTO_CONTRATADO, 
"SISTEMA_ORIGEN, 
"FECHA_DE_ACTUALIZACION

In [126]:
df["PROFESION"] = "N/A"

In [127]:
df['FECHA_DE_ACTUALIZACION'] = datetime.now().strftime('%d/%m/%y')

In [136]:
col_final = ['NEGOCIO', 'NOMBRE_O_RAZON_SOCIAL', 'NUMERO_CLIENTE', 'ESTATUS',
       'NUMERO_CONTRATO', 'GENERO', 'FECHA_NAC_O_CONST',
       'ENTIDAD_FEDERATIVA_NACIMIENTO', 'PAIS_DE_NACIMIENTO', 'NACIONALIDAD',
       'PROFESION', 'CALLE', 'NUMERO_EXTERIOR', 'NUMERO_INTERIOR',
       'COLONIA_URBANIZACION', 'DELEGACION_MUNICIPIO', 'CIUDAD_POBLACION',
       'ENTIDAD_FEDERATIVA', 'CODIGO_POSTAL', 'PAIS', 'TELEFONO',
       'CORREO_ELECTRONICO', 'RFC', 'CURP', 'FIEL', 'REPRESENTANTE_LEGAL',
       'TIPO_PERSONA', 'TIPO_DE_CLIENTE', 'PRODUCTO_CONTRATADO',
       'SISTEMA_ORIGEN', 'FECHA_DE_ACTUALIZACION']


In [138]:
df = df[col_final]

In [139]:
df

,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,NACIONALIDAD,...,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,TIPO_DE_CLIENTE,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
0,NaN,ABA DIVISAS CASA DE CAMBIO SA CV (GUAD),ABADIV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,06/12/24
1,NaN,"ABA DIVISAS CASA DE CAMBIO, SA DE CV",ABA,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,06/12/24
2,UNIDAD DE MERCADO DE CAPITALES,ACCIONES Y VALORES DE MEXICO,ACCIV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,AVM760909UK0,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,06/12/24
3,UNIDAD DE MERCADO DE CAPITALES,ACTINVER CASA DE BOLSA,ACTINV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,ACB7609076M2,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,06/12/24
4,NaN,AMRO BANK MEXICO,AMRO,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,06/12/24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,NaN,"VECTODIVISAS CASA DE CAMBIO,SA DE CV",VECTO,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,06/12/24
259,UNIDAD DE MERCADO DE CAPITALES,"VECTOR, CASA DE BOLSA, S.A DECV",VECT,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXX123456XXX,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,06/12/24
260,NaN,VECTORMEX,VECTOR,VIGENTE,NaN,N/A,NaN,NaN,NaN,EXTRANJERA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,06/12/24
261,NaN,VER FCMX,FCHINY,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,06/12/24


In [130]:
df.columns

Index(['NEGOCIO', 'NOMBRE_O_RAZON_SOCIAL', 'NUMERO_CLIENTE', 'ESTATUS',
       'NUMERO_CONTRATO', 'GENERO', 'FECHA_NAC_O_CONST',
       'ENTIDAD_FEDERATIVA_NACIMIENTO', 'PAIS_DE_NACIMIENTO', 'NACIONALIDAD',
       'PROFESION', 'CALLE', 'NUMERO_EXTERIOR', 'NUMERO_INTERIOR',
       'COLONIA_URBANIZACION', 'DELEGACION_MUNICIPIO', 'CIUDAD_POBLACION',
       'ENTIDAD_FEDERATIVA', 'CODIGO_POSTAL', 'PAIS', 'TELEFONO',
       'CORREO_ELECTRONICO', 'RFC', 'CURP', 'FIEL', 'REPRESENTANTE_LEGAL',
       'TIPO_PERSONA', 'TIPO_DE_CLIENTE', 'PRODUCTO_CONTRATADO',
       'SISTEMA_ORIGEN', 'FECHA_DE_ACTUALIZACION'],
      dtype='object')

In [ ]:
palabras_no_validas = ["PENDIENTE", "JHGJHG", "NINGUNA", "TEL"]

df["CALLE"] = df["CALLE"].apply(
    lambda x: x if isinstance(x, str) and 
               not any(palabra in x for palabra in palabras_no_validas) and \
               x != 'P' and \
               not x.replace(" ", "").isdigit() 
               else np.nan
)

In [140]:
clt_repetidos = compara_nombres(df, df, "NOMBRE_O_RAZON_SOCIAL", "NOMBRE_O_RAZON_SOCIAL","NUMERO_CLIENTE","NUMERO_CLIENTE", 95)
df_resul, drops_ids = agregar_asociaciones_clientes(clt_repetidos,df)
df = df.drop(drops_ids)
df = pd.concat([df, df_resul], ignore_index=True)
df = df.sort_values("NOMBRE_O_RAZON_SOCIAL")

/home/ale1726/proyects/datalake/clientes/clientes_eda/utils/clt_repetidos.py:182: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_resul = df_resul.replace("-", np.nan)


In [141]:
df

,NEGOCIO,NOMBRE_O_RAZON_SOCIAL,NUMERO_CLIENTE,ESTATUS,NUMERO_CONTRATO,GENERO,FECHA_NAC_O_CONST,ENTIDAD_FEDERATIVA_NACIMIENTO,PAIS_DE_NACIMIENTO,NACIONALIDAD,...,CORREO_ELECTRONICO,RFC,CURP,FIEL,REPRESENTANTE_LEGAL,TIPO_PERSONA,TIPO_DE_CLIENTE,PRODUCTO_CONTRATADO,SISTEMA_ORIGEN,FECHA_DE_ACTUALIZACION
0,NaN,ABA DIVISAS CASA DE CAMBIO SA CV (GUAD),ABADIV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,06/12/24
1,NaN,"ABA DIVISAS CASA DE CAMBIO, SA DE CV",ABA,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,06/12/24
2,UNIDAD DE MERCADO DE CAPITALES,ACCIONES Y VALORES DE MEXICO,ACCIV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,AVM760909UK0,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,06/12/24
3,UNIDAD DE MERCADO DE CAPITALES,ACTINVER CASA DE BOLSA,ACTINV,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,ACB7609076M2,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,06/12/24
4,NaN,AMRO BANK MEXICO,AMRO,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,06/12/24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,NaN,"VECTODIVISAS CASA DE CAMBIO,SA DE CV",VECTO,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,CASA DE CAMBIO,NaN,MECA,06/12/24
245,UNIDAD DE MERCADO DE CAPITALES,"VECTOR, CASA DE BOLSA, S.A DECV",VECT,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXX123456XXX,NaN,NaN,NaN,PM,CASA DE BOLSA,NaN,MECA,06/12/24
246,NaN,VECTORMEX,VECTOR,VIGENTE,NaN,N/A,NaN,NaN,NaN,EXTRANJERA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,06/12/24
247,NaN,VER FCMX,FCHINY,VIGENTE,NaN,N/A,NaN,NaN,NaN,MEXICANA,...,NaN,XXXXXXXXXXXXX,NaN,NaN,NaN,PM,BANCO,NaN,MECA,06/12/24


In [ ]:
name_archivo = os.path.join("/home/ale1726/proyects/datalake/clientes/data/ETL","CLientesT_MECA.csv")
df.to_csv(name_archivo,index=False)


In [147]:
df.to_csv(name_archivo,index=False)